In [7]:
import sys
sys.path.insert(0, '..')
import utils.game_utils as gu

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


def get_previous_record(year, week, team, all_games_df):
    records = all_games_df[(all_games_df['year'] == year)
                           & (all_games_df['week'] < week) &
                           (all_games_df['team'] == team)]
    if len(records) < 1:
        return None

    return records[-1:]


def get_all_previous_records(year, week, team, all_games_df):
    records = all_games_df[(all_games_df['year'] == year)
                           & (all_games_df['week'] < week) &
                           (all_games_df['team'] == team)]
    if len(records) < 1:
        return None

    return records


In [8]:
all_games_df = pd.read_csv('../../data/games/all_games_with_data.csv')


def get_df_with_aggregates(index, all_games_df):
    game_df = all_games_df.iloc[index]
    year = game_df['year']
    week = game_df['week']
    team = game_df['team']

    wins = 0
    losses = 0
    ties = 0
    record_normal = 0
    schedule_strength = 0

    previous_df = get_previous_record(year, week, team, all_games_df)


    # info from this game
    win = bool(game_df['team_win'])
    tie = game_df['team_score'] == game_df['opponent_score']
    loss = win == False and tie == False

    def get_val_or(df, field):
        val = df.iloc[0][field]
        if np.isnan(val):
            return 0
        return val

    # get data from previous data
    if previous_df is not None:
        try:
            wins = get_val_or(previous_df, 'team_wins')
            losses = get_val_or(previous_df, 'team_losses')
            ties = get_val_or(previous_df, 'team_ties')
            record_normal = get_val_or(previous_df, 'team_record_normal')
            schedule_strength = get_val_or(previous_df, 'team_schedule_strength')
        except Exception as e:
            print(e)

    if win:
        wins += 1
    if tie:
        ties += 1
    if loss:
        losses += 1

    all_games_df.at[index, [
        'team_wins', 'team_losses', 'team_ties', 'team_record_normal',
        'team_schedule_strength'
    ]] = [wins, losses, ties, record_normal, schedule_strength]

    return all_games_df.iloc[index]


all_games_df = pd.read_csv('../../data/games/all_games_with_data.csv')

## ADD NEW COLUMNS TO ALL GAMES
## AND CREATE NEW ALL_GAMES_PLUS_DF (agp_df)
new_col_df = pd.DataFrame(columns=[
    'team_wins', 'team_losses', 'team_ties', 'team_record_normal',
    'team_schedule_strength'
])
agp_df = all_games_df.join(new_col_df, how="outer")


indexes = [5126,5175,5201,5222,5250,5287,5308,5364,5397]
for index in indexes:
    get_df_with_aggregates(index, agp_df)

# get_df_with_aggregates(5287, agp_df)
# get_df_with_aggregates(5308, agp_df)

test_df = agp_df.iloc[5420]
get_all_previous_records(
    test_df['year'], test_df['week'], test_df['team'],
    agp_df)[['year', 'week', 'team', 'team_wins', 'team_losses', 'team_ties']]


,year,week,team,team_wins,team_losses,team_ties
5126,2020,1,Washington Football Team,1,0,0
5175,2020,2,Washington Football Team,1,1,0
5201,2020,3,Washington Football Team,1,2,0
5222,2020,4,Washington Football Team,1,3,0
5250,2020,5,Washington Football Team,1,4,0
5287,2020,6,Washington Football Team,1,5,0
5308,2020,7,Washington Football Team,2,5,0
5364,2020,9,Washington Football Team,2,6,0
5397,2020,10,Washington Football Team,2,7,0
